In [ ]:
# 1. Imports y configuración
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo visual
sns.set(style="whitegrid")
%matplotlib inline

# 2. Cargar datos
df = pd.read_csv("../data/diabetes.csv")
df.head()

In [ ]:
# 3. Información general del dataset
df.info()
df.describe()
df.isnull().sum()
df.duplicated().sum()

## Hallazgos iniciales

- El dataset contiene 768 registros y 9 variables.
- No hay valores nulos explícitos en ninguna columna.
- Se detectan valores mínimos de cero en variables clínicas (`Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, `BMI`), lo cual es poco realista y será tratado como valor faltante en el preprocesamiento.


In [ ]:
# 4. Distribución de la variable objetivo
sns.countplot(data=df, x="Outcome")
plt.title("Distribución de la variable objetivo (Diabetes)")
plt.xlabel("0 = No Diabetes, 1 = Diabetes")
plt.ylabel("Número de pacientes")

## Distribución de la variable objetivo

- El dataset está desbalanceado: 500 negativos (no diabetes) y 268 positivos (diabetes).
- Este desbalance será considerado durante el modelado, priorizando métricas como F1-score y recall.


In [ ]:
# 5. Visualización de correlaciones
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación entre variables")

## Análisis de correlaciones

- La glucosa (`Glucose`) es la variable más correlacionada con el diagnóstico de diabetes (`Outcome`), con un coeficiente de 0.47.
- Otras variables como IMC (`BMI`), edad (`Age`) y embarazos (`Pregnancies`) también tienen correlaciones positivas con el diagnóstico, pero menores.
- No se detectan pares de variables independientes con correlación mayor a 0.8, por lo que **no existe evidencia de multicolinealidad severa**. Esto es positivo, ya que permite que los modelos aprovechen toda la información de las variables sin redundancia significativa ni riesgos de inestabilidad en los coeficientes.


In [ ]:
# 6. Histogramas de las variables
df.hist(figsize=(15,10), bins=20, edgecolor='black')
plt.suptitle("Distribución de variables", fontsize=16)

In [ ]:
# 7. Boxplots para detectar outliers
features = df.columns[:-1]  # Excluye la columna 'Outcome'
plt.figure(figsize=(15, 10))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i+1)
    sns.boxplot(x=df[feature])
    plt.title(f"Boxplot de {feature}")
plt.tight_layout()

## Outliers y valores extremos

- Se identifican algunos valores atípicos en `Insulin` y `SkinThickness`.
- Estos outliers serán revisados en el preprocesamiento para decidir si deben ser tratados o conservados.


In [ ]:
# 8. Conclusiones iniciales
print("✅ EDA completado. Se identificaron posibles outliers y la clase está desbalanceada.")

## Conclusiones del análisis exploratorio

- El dataset requiere preprocesamiento, especialmente para tratar valores faltantes implícitos (ceros) y posibles outliers.
- El desbalance de clases y la importancia de ciertas variables deben ser considerados en la etapa de modelado.